<a href="https://colab.research.google.com/github/PRAFULPAWAR8888/Agentic_Ai_Projects-Practice/blob/main/WebChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
!pip install PyPDF2

In [89]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
from openai import OpenAI
from google.colab import userdata

import os
from PyPDF2 import PdfReader
import gradio as gr


In [67]:
userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [68]:

client = OpenAI(
    api_key=userdata.get('GOOGLE_API_KEY'),
    base_url="https://generativelanguage.googleapis.com/v1beta"
)

In [69]:
response = client.chat.completions.create(
    model="gemini-2.0-flash",
    messages=[{"role": "user", "content": "Hello, this is a test!"}]
)
print(response)

ChatCompletion(id='-dUqaaKVD4e5vr0P7o6aqAk', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Okay! Consider this test received. Is there anything specific you want me to do with it? Do you have any questions you'd like me to answer or any tasks you'd like me to perform? Just let me know!\n", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1764414970, model='gemini-2.0-flash', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=48, prompt_tokens=7, total_tokens=55, completion_tokens_details=None, prompt_tokens_details=None))


In [70]:
reader = PdfReader("/content/drive/MyDrive/AgenticData/Chatbot/Profile.pdf")
linkedin= ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [71]:
print(linkedin)

   
Contact
At: Bhadawade Post: Shivajinagar
Tal: Khandala Dist: Satara State:
Maharashtra
+91 9322295706  (Mobile)
pawarpraful950@gmail.com
www.linkedin.com/in/praful-
pawar-263526223  (LinkedIn)
github.com/PRAFULPAWAR8888
(Other)
Top Skills
Core Java
MySQL
PHP
Languages
Marathi  (Full Professional)
Hindi  (Full Professional)
English  (Professional Working)
Certifications
Robotics and Artificial Intelligence
Workshop
Career Essentials in Generative AI
by Microsoft and LinkedIn
Career Essentials in Data Analysis
by Microsoft and LinkedIn
Cloud Computing  With
AI( Internship)
DRDO AI /ML ProfestionalPraful Pawar
Artificial Intelligence | Machine learning | Android Development | DSA
| Agentic AI | Data Processing | RAG
Pune, Maharashtra, India
Summary
I am an Artificial Intelligence and Data Science Engineer with a
strong passion for AI, Machine Learning, and Data-Driven problem-
solving. I have hands-on experience in data preprocessing, model
building, fine-tuning, and deploying ML solu

In [72]:
with open("/content/drive/MyDrive/AgenticData/Chatbot/summary.txt","r", encoding="utf-8") as f:
    summary = f.read()

In [73]:
name = "Praful Pawar"

In [74]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [75]:
system_prompt

'You are acting as Praful Pawar. You are answering questions on Praful Pawar\'s website, particularly questions related to Praful Pawar\'s career, background, skills and experience. Your responsibility is to represent Praful Pawar for interactions on the website as faithfully as possible. You are given a summary of Praful Pawar\'s background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don\'t know the answer, say so.\n\n## Summary:\nMotivated and detail-oriented AI & Data Science graduate with hands-on experience in machine learning, deep learning, and data analysis. Skilled in building AI/ML projects using TensorFlow, PyTorch, scikit-learn, and NumPy. Experienced in developing end-to-end solutions, including data preprocessing, model training, evaluation, and deployment. Passionate about GenAI, LLMs, and AI-powered automation, with practical exposure 

In [76]:
def chat(message, history):
  messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
  response = client.chat.completions.create(
    model="gemini-2.0-flash",
    messages=messages)
  return response.choices[0].message.content

In [77]:
gr.ChatInterface(chat, type="messages").launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8ebe2ada03bd3faa63.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# A lot is abotut happen

*   Be able to ask LLM to evaluate an answer
*   Be able to return if any answer fails to return
*   Pull this  togeter into one workflow



In [78]:
# creaete a pydentic model for evaluation
from pydantic import BaseModel

class Evaluation(BaseModel):
  if_acceptable: bool
  feedback: str

In [79]:
evaluator_system_prompt = (
    f"You are an evaluator responsible for determining whether the Agent's latest response "
    f"is acceptable. You are given a conversation between a User and an Agent. Your task is "
    f"to assess whether the Agent's most recent reply aligns with the expected behavior.\n\n"

    f"The Agent is playing the role of {name} and is representing {name} on their website. "
    f"The Agent has been instructed to maintain a professional and engaging tone, as if "
    f"interacting with potential clients or future employers visiting the site.\n\n"

    f"The Agent has also been provided with contextual information about {name}, including "
    f"their summary and LinkedIn details. Here is the information:\n\n"

    f"## Summary:\n{summary}\n\n"
    f"## LinkedIn Profile:\n{linkedin}\n\n"

    f"Using this context, please evaluate the Agent's latest response and state whether the "
    f"response is acceptable. Provide a clear explanation for your evaluation."
)



In [80]:
def evaluator_user_prompt(reply, message, history):
  user_prompt = f"Here is the conversation between the User and the Agents: \n\n{history}\n\n"
  user_prompt = f"Here is the latest message from the  User: \n\n{message}\n\n"
  user_prompt = f"Here is the latest response form the agent:\n\n{reply}\n\n"
  user_prompt = f"Please evaluate the reponse, replying with whether it is acceptable and your feedback."
  return user_prompt

In [81]:
import os
client = OpenAI(
    api_key=userdata.get('GOOGLE_API_KEY'),
    base_url="https://generativelanguage.googleapis.com/v1beta"
)

In [82]:
def evaluate(reply, message, history) -> Evaluation:
  messages = [{"role":"system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply,message,history)}]
  response = client.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format =Evaluation)
  return response.choices[0].message.parsed

In [83]:
messages = [{"role":"system", "content": evaluator_system_prompt}] + [{"role": "user", "content":"do you hold a patent?"}]
response = client.beta.chat.completions.parse(model="gemini-2.0-flash",messages=messages)
reply = response.choices[0].message.content


In [84]:
reply


"As a recent graduate, I don't currently hold any patents. However, I am eager to contribute to innovative projects and develop solutions that could potentially lead to patentable inventions in the future. I am always learning and exploring new ideas, and I am excited about the prospect of contributing to cutting-edge research and development.\n"

In [85]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(if_acceptable=True, feedback="The prompt is a meta-prompt asking me to evaluate the Agent's response. Since I don't have the Agent's response, I cannot evaluate it. However, I can confirm that the structure of this response adheres to the required JSON format, and would be deemed acceptable if there was an agent response to evaluate.")

In [86]:
def rerun(reply, message, feedback, history):
  updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\n You just tried to reply, but the quality control rejected your"
  updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
  updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
  messages = [{"role": "system", "content": updated_system_prompt}] +history+[{"role": "user", "content": message}]
  response = client.chat.completions.create(model="gemini-2.0-flash", messages=messages)
  return response.choices[0].message.content


In [92]:
def chat(message, history):
  system = system_prompt
  messages = [{"role": "system", "content": system}] + history+ [{"role":"user", "content": message}]
  response = client.chat.completions.create(model = "gemini-2.0-flash", messages=messages)
  reply = response.choices[0].message.content

  evaluation = evaluate(reply, message, history)
  if evaluation.if_acceptable:
    print("Passed evaluation - returning reply")
  else:
    print("Failed evaluation - retrying")
    print(evaluation.feedback)
    reply = rerun(reply, message, evaluation.feedback ,history)
  return reply

In [93]:
gr.ChatInterface(chat, type="messages").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://19c2fb05032141a0a0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [87]:
 Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://19c2fb05032141a0a0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
